# Reusing and bundling

To standardize our environment, we follow this road:

- define reusable and versioned components (eg on github)
- `$ref`erence reusable components
- eventualy create bundles (ewg. ith `openapi_resolver`)


### Exercise: replacing definitions with $refs

Starting from a complete file [ex-05-01-bundle.yaml](ex-05-01-bundle.yaml)
replace as many definitions as possible with references from the shared  [definitions.yaml](https://teamdigitale.github.io/openapi/0.0.5/definitions.yaml).



In [ ]:
# Exercise: creating a bundle from a $ref file
# 
# You can resolve dependencies and create a bundle file with
!pip install openapi_resolver


In [ ]:
# Exercise: create a bundle from the previous file with
!python -m openapi_resolver /code/notebooks/oas3/ex-05-01-bundle.yaml

### YAML anchors are your friends

YAML has a very nice feature, named **anchors**. They allows to define and reference
given portions of a YAML file.

```
# the following &anchor stores the `foo` value 
a: &this_is_an_anchor foo

# *star dereferences the anchor
b: *this_is_an_anchor

```

See [anchors.yaml](/edit/notebooks/anchors.yaml)

In [ ]:
#  Check yaml file content.
from pathlib import Path

content = Path('anchors.yaml').read_text()

print(content)

In [ ]:
ret = yaml.safe_load(content)

assert ret['anchored_content'], ret['other_anchor']

print(ret['foo'])
print(ret['bar'])

### Using YAML anchors in OAS3

As every operation may have a set of predefined response, namely:

- 503 Service Unavailable
- 429 Too Many Requests

You can put them in an `x-` custom parameter.which will be ignored by the OAS spec parser. 

```
x-common-responses: &common-responses
  503ServiceUnavailable:
    $ref: ... 
  429TooManyRequests:
    $ref: ...

```

Then use the anchor to reference them.

```
paths:
  /status:
    get:
      ...
      responses:
        # Add the anchored responses
        <<: *common-responses
        
        # And now the other ones.
        '200':
          ...
```

## NOTE: ANCHORS ARE PROCESSED BY THE YAML PARSER, NOT BY OAS
## OAS knows nothing about ANCHORS